## Funcion Guardar Fotos de un Video

In [3]:
import cv2
import os

def save_frames_from_video(video_path, output_folder, frame_size=(100, 100), frame_interval=10):
    os.makedirs(output_folder, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        resized_frame = cv2.resize(frame, frame_size)

        frame_filename = f"frame_{frame_count:04d}.jpg"
        frame_path = os.path.join(output_folder, frame_filename)
        cv2.imwrite(frame_path, resized_frame)

        frame_count += 1

        # Intervalo
        for _ in range(frame_interval):
            ret, _ = cap.read()
            if not ret:
                break

    print(f"Fotos guardadas en '{output_folder}'")
    cap.release()


In [ ]:
pfolder = './p'
nfolder = './n'
pvideo = './positive.mp4'
nvideo = './negative.mp4'

if not os.path.exists(pfolder):
    os.makedirs(pfolder)
    print(f"Folder '{pfolder}' creado")

if not os.path.exists(nfolder):
    os.makedirs(nfolder)
    print(f"Folder '{nfolder}' creado")

# save_frames_from_video(nvideo, nfolder, frame_size=(100, 100), frame_interval=10)
# save_frames_from_video(pvideo, pfolder, frame_size=(100, 100), frame_interval=10)